In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
df = pd.read_csv('Sources/fiuba_6_avisos_detalle_concat.csv').drop(['Unnamed: 0'],axis=1)
cluster = pd.read_csv('Sources/idaviso-cluster.csv').drop('Unnamed: 0',axis=1)
features = pd.read_csv('Sources/mas_features.csv').drop('Unnamed: 0',axis=1)
tuples = [tuple(x) for x in df.values]
df.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [4]:
for x in tuples:
    x = tuple(x)

In [5]:
avisos = sc.parallelize(tuples)

In [6]:
avisos = avisos.map(lambda x: (x[0],x[1],x[2],str.upper(re.sub("<.*?>", " ", x[3])),str.upper(re.sub("<.*?>", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10])).map(lambda x: (x[0],x[1],x[2],str(re.sub("\xa0", " ", x[3])),str(re.sub("\xa0", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10]))

In [7]:
import re, string, unicodedata
def removePunctuation(text):
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

In [8]:
def palabrasLargas(texto):
    lista = texto.split(' ') 
    lista = [x for x in lista if not len(x)>17]
    return " ".join(lista)

In [9]:
avisos = avisos.map((lambda x: (x[0],x[1],x[2],removePunctuation(x[3]),removePunctuation(x[4]),x[5],x[6],x[7],x[8],x[9],x[10]))).map((lambda x: (x[0],x[1],x[2],palabrasLargas(x[3]),x[4],x[5],x[6],x[7],x[8], x[9],x[10]))).collect()

In [10]:
avisos = pd.DataFrame.from_records(avisos)
avisos.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,buscamos un responsable de producto para inc...,gran buenos aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,para nuestro cliente concesionario chevrolet u...,gran buenos aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,importante estudio de auditoria y consultoria ...,gran buenos aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,chevron is a multinational company in the ener...,gran buenos aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,importante empresa de energias renovables se e...,gran buenos aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [13]:
for x in avisos.columns:  
    avisos = avisos.rename(columns = {x:df.columns[x]})
avisos.head()
avisos = avisos.drop(['idpais','nombre_zona','mapacalle', 'ciudad'], axis=1)
avisos.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,buscamos un responsable de producto para inc...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,para nuestro cliente concesionario chevrolet u...,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,importante estudio de auditoria y consultoria ...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,chevron is a multinational company in the ener...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,importante empresa de energias renovables se e...,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [14]:
avisos = avisos.merge(cluster,on='idaviso',how='left')
avisos.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster
0,1111556097,Responsable de Marketing-Producto Div. Constru...,buscamos un responsable de producto para inc...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,109
1,1111949392,Asesor de Servicios - Postventa Chevrolet,para nuestro cliente concesionario chevrolet u...,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,3
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,importante estudio de auditoria y consultoria ...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,161
3,1112146010,Software Engineer,chevron is a multinational company in the ener...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,79
4,1112211475,Business Developer Manager,importante empresa de energias renovables se e...,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,1


In [15]:
avisos = avisos.merge(features, on='idaviso',how='left')
avisos.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad
0,1111556097,Responsable de Marketing-Producto Div. Constru...,buscamos un responsable de producto para inc...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,109,96.0,129.0,2.0,MASC,32.266667
1,1111949392,Asesor de Servicios - Postventa Chevrolet,para nuestro cliente concesionario chevrolet u...,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,3,31.0,155.0,1.0,MASC,32.032258
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,importante estudio de auditoria y consultoria ...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,161,61.0,65.0,0.0,NaN,29.650794
3,1112146010,Software Engineer,chevron is a multinational company in the ener...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,79,6.0,42.0,3.0,MASC,30.541667
4,1112211475,Business Developer Manager,importante empresa de energias renovables se e...,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,1,0.0,6.0,0.0,MASC,30.166667


In [16]:
avisos.to_csv("Sources/fiuba_6_avisos_detalle_limpio.csv")